In [ ]:
import pandas as pd
import tensorflow as tf
import multiprocessing
import editdistance

In [ ]:
navn = tf.feature_column.numeric_column("r_name")
fødested = tf.feature_column.numeric_column("r_birthp")
civilstand = tf.feature_column.numeric_column("r_civil")
position = tf.feature_column.numeric_column("r_pos")
erhverv = tf.feature_column.numeric_column("r_job")
køn = tf.feature_column.numeric_column("m_gender")
fødeår = tf.feature_column.numeric_column("d_birthy")

In [ ]:
model2 = tf.estimator.LinearClassifier(feature_columns=[
    navn, fødested, civilstand, position, erhverv, køn, fødeår
], model_dir="model")

In [ ]:
df = pd.read_pickle("m-TT.pickled")

In [ ]:
len(df)

In [ ]:
import recordlinkage

In [ ]:
dfA = df[df.FT==1845]

In [ ]:
len(dfA)

In [ ]:
dfB = df[df.FT==1850]

In [ ]:
len(dfB)

In [ ]:
len(dfA) * len(dfB)

In [ ]:
indexer = recordlinkage.indexing.SortedNeighbourhoodIndex(on="Fødeår", window=5)

In [ ]:
indexer = recordlinkage.indexing.FullIndex()

In [ ]:
index = indexer.index(dfA, dfB)

In [ ]:
len(index)

In [ ]:
def ss(a: str, b: str):
    """Similarity score for two strings a and b"""
    if pd.isnull(a) or pd.isnull(b):
        return 0
    if a==b:
        return 1.0
    return editdistance.eval(a, b) / max(len(a), len(b))

In [ ]:
def score(a, b):
    return (ss(a.Navn, b.Navn),
            ss(a.Fødested, b.Fødested),
            ss(a.Civilstand, b.Civilstand),
            ss(a.Position, b.Position),
            ss(a.Erhverv, b.Erhverv),
            1.0 if a.Køn == b.Køn else 0.0,
            abs(a.Fødeår - b.Fødeår))

In [ ]:
score_columns="r_name r_birthp r_civil r_pos r_job m_gender d_birthy".split(" ")

In [ ]:
def generate_scores(d):
    lots = []
    for (a, b) in d:
        a = dfA.loc[a]
        b = dfB.loc[b]
        lots.append((a.FT, a.Kipnr, a.Løbenr, b.FT, b.Kipnr, b.Løbenr) + score(a,b))
    return lots

In [ ]:
import utils

In [ ]:
with multiprocessing.Pool() as p:
    res = p.map(generate_scores, utils.chunks(index, 1000))

In [ ]:
import itertools

In [ ]:
df_eval = pd.DataFrame(list(itertools.chain(*res)), columns=["a_FT", "a_Kipnr", "a_Løbenr", "b_FT", "b_Kipnr", "b_Løbenr"] + score_columns)

In [ ]:
input_fn = tf.estimator.inputs.pandas_input_fn(x=df_eval, shuffle=False)

In [ ]:
prediction = list(model2.predict(input_fn))

In [ ]:
df.set_index(["FT", "Kipnr", "Løbenr"], inplace=True)

In [ ]:
df.sort_index(inplace=True)

In [ ]:
len(prediction)

In [ ]:
header = "Amt|Herred|Sogn|Navn|Køn|Fødested|Fødeår|Civilstand|Position|Erhverv|Kipnr|Løbenr|Group".split("|")

In [ ]:
for i, d in enumerate(prediction):
    if d["probabilities"][0] < 0.135:
        print(d["probabilities"])
        print(d["classes"])
        r = df_eval.iloc[i]
        s = df.loc[[(r.a_FT,r.a_Kipnr,r.a_Løbenr),
                     (r.b_FT,r.b_Kipnr,r.b_Løbenr)]]
        a = s.iloc[0]
        b = s.iloc[1]
        for i in range(11):
            print("{:10s} {:40s} {:40s}".format(header[i], str(a[i]), str(b[i])))
        print()

In [ ]:
d

In [ ]:
model2.get_variable_names()

In [ ]:
for var in model2.get_variable_names():
    print(var)
    print(model2.get_variable_value(var))